In [1]:
from nas.supernet.supernet_yolov7 import YOLOSuperNet
from latency_predictor.arch_utils import *
import time

device = 'cpu'
supernet = YOLOSuperNet(cfg='./yaml/yolov7_dynamicsupernet.yml').to(device)
supernet.train()

backbone_lut = {}
head_lut = {}

# cal. latency for backbone
# for i in range(1,6):
#     for j in range(1,6):
#         for k in range(1,6):
#             for m in range(1,6):
#                 supernet.set_active_subnet([i,j,k,m,1,1,1,1]) # Maximum: [5,5,5,5,7,7,7,7]
#                 subnet = supernet.get_active_subnet()

#                 b_arch, _, save = split_backbone_head(subnet)

#                 #########이부분이 galaxy 측정으로 대체되어야 함
#                 backbone = skin_backbone(b_arch, save)
#                 img = torch.rand(1, 3, 640, 640).to(device)
#                 s = time.time()
#                 y = backbone(img)
#                 latency = (time.time()-s )*1000 # ms :: 실제로 측정할때는 1000 번이상 측정해서 평균값으로

#                 backbone_lut[str(i)+str(j)+str(k)+str(m)] = latency

# print("backbone completed")

# cal. latency for head
for i in range(1,8):
    for j in range(1,8):
        for k in range(1,8):
            for m in range(1,8):
                supernet.set_active_subnet([1,1,1,1,i,j,k,m]) # Maximum: [5,5,5,5,7,7,7,7]
                subnet = supernet.get_active_subnet()

                _, h_arch, save = split_backbone_head(subnet)

                #########이부분이 galaxy 측정으로 대체되어야 함
                head = skin_head(h_arch, save)
                img = torch.rand(1, 1024, 20, 20).to(device)
                s = time.time()
                y = head(img)
                latency = (time.time()-s )*1000 # ms :: 실제로 측정할때는 1000 번이상 측정해서 평균값으로

                head_lut[str(i)+str(j)+str(k)+str(m)] = latency




d:\codes_ETRI\yolov7-nas-for-TANGO\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

file_path = "latency_predictor/lut/head.json"

with open(file_path, 'w') as f:
    json.dump(head_lut, f)